In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.pandas as pd
import pyspark

/usr/local/spark/python/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
spark = SparkSession.builder \
        .appName("Relational Sample PySpark SQL Server Connection") \
        .config("spark.jars", "../jdbc/mssql-jdbc-12.6.1.jre8.jar") \
        .getOrCreate()

In [3]:
def spark_read_from_db(table_name):
    server_name = "mssql"
    port = "1433"
    database_name = "Data"
    url = f"jdbc:sqlserver://{server_name}:{port};databaseName={database_name}"
    username = "SA"
    password = "YourStrongPassword123"
    df = spark.read \
        .format("jdbc") \
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password) \
        .option("encrypt", "false") \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .load()
    return df

In [6]:
properties = spark_read_from_db("ARProperties")
clients = spark_read_from_db("ARClients")
properties_clients = spark_read_from_db("PropertiesClients")

In [15]:
join_condition_properties = properties["id"] == properties_clients["id_prop"]
join_condition_clients = clients["id"] == properties_clients["id_client"]

joined_df = properties_clients.join(clients, join_condition_clients) \
                              .join(properties, join_condition_properties)
columns_to_drop = ["id"]
joined_df = joined_df.drop(*columns_to_drop)

In [21]:
joined_df.sample(10)

PySparkTypeError: [NOT_BOOL_OR_FLOAT_OR_INT] Argument `withReplacement (optional), fraction (required) and seed (optional)` should be a bool, float or str, got int, NoneType, NoneType.